## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import metrics
import datetime

## Loading data

In [2]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')

In [3]:
features_and_target = pd.read_csv('data/features_and_target.csv', parse_dates=['day'], index_col='day',date_parser=dateparse)

In [4]:
train_test = features_and_target[:'2018-06-28']
validation = features_and_target['2018-06-28':]

train = train_test[:'2018-05-19']
test = train_test['2018-05-19':]

y = train['num_pickups'].values
train = train.drop('num_pickups', axis=1)

y_test = test['num_pickups'].values
test = test.drop('num_pickups', axis=1)

y_val = validation['num_pickups'].values
val = validation.drop('num_pickups', axis=1)

# Use Prophet
to compare results with engineered features

In [5]:
from fbprophet import Prophet

## Predict 7 day  from 2018-07-10 with Prophet

In [6]:
train_prophet2 = features_and_target[:'2018-07-10']
test_prophet2 = features_and_target['2018-07-10':]

In [7]:
y_prophet = train_prophet2['num_pickups'].values
train_prophet = train_prophet2.drop('num_pickups', axis=1)

In [8]:
train_prophet_challenge = pd.DataFrame()
train_prophet_challenge['ds'] = train_prophet.index
train_prophet_challenge['y'] = y_prophet

In [9]:
model_prophet_challenge = Prophet()
model_prophet_challenge.fit(train_prophet_challenge)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [10]:
future_challenge = model_prophet_challenge.make_future_dataframe(periods=7)
future_challenge.tail()

,ds
172,2018-07-13
173,2018-07-14
174,2018-07-15
175,2018-07-16
176,2018-07-17


## Forecast 

In [11]:
forecast_challenge = model_prophet_challenge.predict(future_challenge)
forecast_challenge[['ds', 'yhat']].tail(8)

,ds,yhat
169,2018-07-10,52.744032
170,2018-07-11,65.366679
171,2018-07-12,71.534375
172,2018-07-13,77.154298
173,2018-07-14,43.159931
174,2018-07-15,71.692714
175,2018-07-16,73.741005
176,2018-07-17,53.113489


In [12]:
forecasts_df = pd.DataFrame()
forecasts_df['day'] = forecast_challenge['ds'].tail(8)
forecasts_df['forecast'] =  forecast_challenge['yhat'].tail(8)
forecasts_df.to_csv('data/forecast_prophet.csv', index=False, header=True)